# Implementing a Route Planner
In this project a "Google-maps" style route planner is implemented using A\* search algorithm.

## The Map

In [ ]:
# Run this cell first!

from helpers import Map, load_map_10, load_map_40, show_map
import math

%load_ext autoreload
%autoreload 2

## The Algorithm

### PathPlanner class

The below class is given, and the tast is to implement the methods that are missing to make the PathPlanner() class fully functional.

Walk through on the PathPlanner() class:

`__init__` - Map, starting node and goal node are input and initialized for the class, besides, other attributes and methods needed for the algorithm are also initialized.
- `closedSet` includes any explored/visited nodes. 
- `openSet` are any nodes in frontier for future exploration. 
- `cameFrom` will hold the previous node that's been selected by the A* algorithm.
- `gScore` is the `g` in `f = g + h` equation, or the cost from starting node to the current node under evaluation.
- `fScore` is the combination of `g` and `h`, i.e. the `gScore` plus a heuristic; total cost to reach the goal.
- `path` comes from the `run_search` method, which is the final path found by A* search.

`reconstruct_path` - This function just rebuilds the path after path is found, going from the goal node backwards using each node's `cameFrom` information.

`_reset` - Resets *most* of our initialized variables for PathPlanner. This *does not* reset the map, start or goal variables.

`run_search` - This is where A* search is running. 
- First, it does sanity check to make sure everything needed by the algorithm has been correctly initialized.
- In the main loop, if there is still unexplored nodes, the one with the lowest 'f' score will be chosen (by self.get_current_node()). For the first iteration with only the 'start' node in openSet, this makes sure that the algorithm can start.
- For the selected node, if it's the goal, the path will be re-built and saved, it will be the final output if no better solutions are found, if not, the selected node is put to closedSet and removed from openSet.
- The last part is to update the information for the neighbors of the selected node. There are 3 types of neighbors to the current node: (1)those in the closedSet; (2) those belong to unexplored set; (3)those also in openSet(frontier). Operation on each type is different:
   (1) for closedSet nodes, no action needed, just continue searching;
   (2) for unexplored nodes, add those to openSet for future exploration;
   (3) for openSet nodes, it can be a little bit triky. Let's say the current node is A, which comes from B and C is the neighbor of A that's also in the openSet. Now, C already has a 'g' Score (that's why we can evaluate 'f' score at the beginning and decide that A is better than C), and C may or may not come from B. However, A has been chosen as the current node (meaning for sure, the path will include A), we want to check if from A -> C is better than from (?) -> C. If it does, the 'g' score (meaning the cost from starting node to C) can be improved by chaning the path from (?)-> C to ...-> A -> C, if it does not, then we need to keep the best 'g' score for C, which is (?)->C, and leave this to futher evaluation.



In [ ]:
class PathPlanner():
    """Construct a PathPlanner Object"""
    def __init__(self, M, start=None, goal=None):
        """ """
        self.map = M
        self.start= start
        self.goal = goal
        self.closedSet = self.create_closedSet() if goal != None and start != None else None
        self.openSet = self.create_openSet() if goal != None and start != None else None
        self.cameFrom = self.create_cameFrom() if goal != None and start != None else None
        self.gScore = self.create_gScore() if goal != None and start != None else None
        self.fScore = self.create_fScore() if goal != None and start != None else None
        self.path = self.run_search() if self.map and self.start != None and self.goal != None else None
    
    def reconstruct_path(self, current):
        """ Reconstructs path after search """
        total_path = [current]
        while current in self.cameFrom.keys():
            current = self.cameFrom[current]
            total_path.append(current)
        return total_path
    
    def _reset(self):
        """Private method used to reset the closedSet, openSet, cameFrom, gScore, fScore, and path attributes"""
        self.closedSet = None
        self.openSet = None
        self.cameFrom = None
        self.gScore = None
        self.fScore = None
        self.path = self.run_search() if self.map and self.start and self.goal else None

    def run_search(self):
        """ """
        if self.map == None:
            raise(ValueError, "Must create map before running search. Try running PathPlanner.set_map(start_node)")
        if self.goal == None:
            raise(ValueError, "Must create goal node before running search. Try running PathPlanner.set_goal(start_node)")
        if self.start == None:
            raise(ValueError, "Must create start node before running search. Try running PathPlanner.set_start(start_node)")

        self.closedSet = self.closedSet if self.closedSet != None else self.create_closedSet()
        self.openSet = self.openSet if self.openSet != None else  self.create_openSet()
        self.cameFrom = self.cameFrom if self.cameFrom != None else  self.create_cameFrom()
        self.gScore = self.gScore if self.gScore != None else  self.create_gScore()
        self.fScore = self.fScore if self.fScore != None else  self.create_fScore()

        while not self.is_open_empty():
            current = self.get_current_node()

            if current == self.goal:
                self.path = [x for x in reversed(self.reconstruct_path(current))]
                return self.path
            else:
                self.openSet.remove(current)
                self.closedSet.add(current)

            for neighbor in self.get_neighbors(current):
                if neighbor in self.closedSet:
                    continue    # Ignore the neighbor which is already evaluated.

                if not neighbor in self.openSet:    # Discover a new node
                    self.openSet.add(neighbor)
                
                # The distance from start to a neighbor
                #the "dist_between" function may vary as per the solution requirements.
                if self.get_tentative_gScore(current, neighbor) >= self.get_gScore(neighbor):
                    continue        # This is not a better path.

                # This path is the best until now. Record it!
                self.record_best_path_to(current, neighbor)
        print("No Path Found")
        self.path = None
        return False

### Data Structures

Initialize closedSet and openSet as data type : set() since in the search algorithm, a node can be repeatedly added to closedSet and openSet. To get rid of the redundancy, python set() is used. The openSet has to have 'start' as the initialization for the search algorithm to proceed. 

In [ ]:
def create_closedSet(self):
    """ Creates and returns a data structure suitable to hold the set of nodes already evaluated"""
    # EXAMPLE: return a data structure suitable to hold the set of nodes already evaluated
    return set()

In [ ]:
def create_openSet(self):
    """ Creates and returns a data structure suitable to hold the set of currently discovered nodes 
    that are not evaluated yet. Initially, only the start node is known."""
    if self.start != None:
        # TODO: return a data structure suitable to hold the set of currently discovered nodes 
        # that are not evaluated yet. Make sure to include the start node.
        return set({self.start})
    
    raise(ValueError, "Must create start node before creating an open set. Try running PathPlanner.set_start(start_node)")

cameFrom is defined as a python dictionary, where the key is certain node and the value shows where the preceeding node is.

In [ ]:
def create_cameFrom(self):
    """Creates and returns a data structure that shows which node can most efficiently be reached from another,
    for each node."""
    # TODO: return a data structure that shows which node can most efficiently be reached from another,
    # for each node. 
    
    return {}

By definition, 'g' of start node is 0. The reason to set the rest nodes to 'inf' is to have :  
`if self.get_tentative_gScore(current, neighbor) >= self.get_gScore(neighbor)`  
always false for all the unexplored node.

In [ ]:
def create_gScore(self):
    """Creates and returns a data structure that holds the cost of getting from the start node to that node, 
    for each node. The cost of going from start to start is zero."""
    # TODO:  return a data structure that holds the cost of getting from the start node to that node, for each node.
    # for each node. The cost of going from start to start is zero. The rest of the node's values should 
    # be set to infinity.
    gScore = {}
    for i in self.map.intersections:
        if i == self.start:
            gScore[i] = 0
        else :
            gScore[i] = float("inf")
            
    return gScore

Similar to initialize 'g' score, now the 'f' of start is equal to 'h'

In [ ]:
def create_fScore(self):
    """Creates and returns a data structure that holds the total cost of getting from the start node to the goal
    by passing by that node, for each node. That value is partly known, partly heuristic.
    For the first node, that value is completely heuristic."""
    # TODO: return a data structure that holds the total cost of getting from the start node to the goal
    # by passing by that node, for each node. That value is partly known, partly heuristic.
    # For the first node, that value is completely heuristic. The rest of the node's value should be 
    # set to infinity.
    
    fScore = {}
    for i in self.map.intersections:
        if i == self.start:
            fScore[i] = self.heuristic_cost_estimate(i)
        else :
            fScore[i] = float("inf")
            
    return fScore

### Set certain variables

The below functions help set certain variables if they weren't a part of initializating our `PathPlanner` class, or if they need to be changed for anothe reason.

In [ ]:
def set_map(self, M):
    """Method used to set map attribute """
    self._reset(self)
    # TODO: Set map to new value. 
    self.map = M

In [ ]:
def set_start(self, start):
    """Method used to set start attribute """
    #self._reset(self)
    # TODO: Set start value. Remember to remove goal, closedSet, openSet, cameFrom, gScore, fScore, 
    # and path attributes' values.
    self.start = None
    self._reset(self)
    self.start = start

In [ ]:
def set_goal(self, goal):
    """Method used to set goal attribute """
    self._reset(self)
    # TODO: Set goal value. 
    self.goal = goal

### Get node information

The below functions concern grabbing certain node information. In `is_open_empty`, you are checking whether there are still nodes on the frontier to explore. In `get_current_node()`, you'll want to come up with a way to find the lowest `fScore` of the nodes on the frontier. In `get_neighbors`, you'll need to gather information from the map to find the neighbors of the current node.

In [ ]:
def is_open_empty(self):
    """returns True if the open set is empty. False otherwise. """
    # TODO: Return True if the open set is empty. False otherwise.
    return not bool(self.openSet)

In [ ]:
def get_current_node(self):
    """ Returns the node in the open set with the lowest value of f(node)."""
    # TODO: Return the node in the open set with the lowest value of f(node).
    minF = float("inf")
    currentNode = None
    for node in self.openSet:
        costF = self.fScore[node] 
        if costF < minF :
            minF = costF
            currentNode = node
        else :
            continue
            
    return currentNode

In [ ]:
def get_neighbors(self, node):
    """Returns the neighbors of a node"""
    # TODO: Return the neighbors of a node
    return self.map.roads[node]

### Scores and Costs

Below, you'll get into the main part of the calculation for determining the best path - calculating the various parts of the `fScore`.

In [ ]:
def get_gScore(self, node):
    """Returns the g Score of a node"""
    # TODO: Return the g Score of a node
    return self.gScore[node]

In [ ]:
def distance(self, node_1, node_2):
    """ Computes the Euclidean L2 Distance"""
    # TODO: Compute and return the Euclidean L2 Distance
    return math.sqrt((self.map.intersections[node_1][0] - self.map.intersections[node_2][0])**2 + (self.map.intersections[node_1][1] - self.map.intersections[node_2][1])**2)

In [ ]:
def get_tentative_gScore(self, current, neighbor):
    """Returns the tentative g Score of a node"""
    # TODO: Return the g Score of the current node 
    # plus distance from the current node to it's neighbors
    dist = self.distance(current, neighbor)
    return self.get_gScore(current) + dist

In [ ]:
def heuristic_cost_estimate(self, node):
    """ Returns the heuristic cost estimate of a node """
    # TODO: Return the heuristic cost estimate of a node
    return self.distance(node,self.goal)

In [ ]:
def calculate_fscore(self, node):
    """Calculate the f score of a node. """
    # TODO: Calculate and returns the f score of a node. 
    # REMEMBER F = G + H
    return self.gScore[node] + self.heuristic_cost_estimate(node)

### Recording the best path

Now that you've implemented the various functions on scoring, you can record the best path to a given neighbor node from the current node!

In [ ]:
def record_best_path_to(self, current, neighbor):
    """Record the best path to a node """
    # TODO: Record the best path to a node, by updating cameFrom, gScore, and fScore
    self.cameFrom[neighbor] = current
    self.gScore[neighbor] = self.get_tentative_gScore(current,neighbor)
    self.fScore[neighbor] = self.calculate_fscore(neighbor)

### Associating your functions with the `PathPlanner` class

To check the implementations, associate all of the above functions back to the `PathPlanner` class. Python makes this easy using the dot notation (i.e. `PathPlanner.myFunction`), and setting them equal to the function implementations.

In [ ]:
# Associates implemented functions with PathPlanner class
PathPlanner.create_closedSet = create_closedSet
PathPlanner.create_openSet = create_openSet
PathPlanner.create_cameFrom = create_cameFrom
PathPlanner.create_gScore = create_gScore
PathPlanner.create_fScore = create_fScore
PathPlanner.set_map = set_map
PathPlanner.set_start = set_start
PathPlanner.set_goal = set_goal
PathPlanner.is_open_empty = is_open_empty
PathPlanner.get_current_node = get_current_node
PathPlanner.get_neighbors = get_neighbors
PathPlanner.get_gScore = get_gScore
PathPlanner.distance = distance
PathPlanner.get_tentative_gScore = get_tentative_gScore
PathPlanner.heuristic_cost_estimate = heuristic_cost_estimate
PathPlanner.calculate_fscore = calculate_fscore
PathPlanner.record_best_path_to = record_best_path_to

### Preliminary Test

The below is the first test case, just based off of one set of inputs. If some of the functions above aren't implemented yet, or are implemented incorrectly, you likely will get an error from running this cell. Try debugging the error to help you figure out what needs further revision!

In [ ]:
planner = PathPlanner(map_40, 5, 34)
path = planner.path
if path == [5, 16, 37, 12, 34]:
    print("great! Your code works for these inputs!")
else:
    print("something is off, your code produced the following:")
    print(path)

---